<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #6495ed; margin-bottom:10px;">
  <h4 style="font-size:13px;">📦 <strong>Importar Librerias</strong></h4>
</div>

In [ ]:
import ray
from ray import serve, init
import mlflow
import pandas as pd
from fastapi import FastAPI
from fastapi import Request
from pydantic import BaseModel
from fastapi.exceptions import RequestValidationError
from fastapi.responses import JSONResponse
from starlette.requests import Request
from json import JSONDecodeError
from sklearn.datasets import load_iris
import requests

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Conecta a un cluster de Ray ya iniciado</strong></h4>
</div>

In [ ]:
try:
    ray_head = "ray-head"
    ray.init(address=f'ray://{ray_head}:10001') #SERVER
    serve.start(detached=True)
    print("Conexion correcta al servicio Ray ...")
except Exception as ex:
    print(f"Ray: Se produjo el siguiente error : {ex} ")

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Configuracion del Modelo</strong></h4>
</div>

In [ ]:
try:
    app = FastAPI(
        title = "Import Predictor API",
        description = "Import Pipeline online inference"
    )

    @app.exception_handler(RequestValidationError)
    async def validation_exception_handler(request: Request, exc: RequestValidationError):
        mensajes = [err["msg"] for err in exc.errors()]
        return JSONResponse(
            status_code = 422,
            content = { "cod": 422, "msg": "; ".join(mensajes), "data": None, "errors": exc.errors(), },
        )
    
    class Item(BaseModel):
        sepal_length : float = 6.6
        sepal_width  : float = 3.0
        petal_length : float = 4.4
        petal_width  : float = 1.4

    @serve.deployment(
        route_prefix="/v1/iris-logreg-modelo/score",
        ray_actor_options={"num_cpus": 0.1, "num_gpus": 0},
        name = "Iris_LogReg_Model"
    )
    @serve.ingress(app)
    class ModelMlFlow:
        def __init__(self):
            #mlflow.set_tracking_uri("http://172.26.56.133:5000")
            model_uri = "models:/Iris_LogReg_Model/Production"
            try:
                self.model = mlflow.sklearn.load_model(model_uri)
            except Exception as ex:
                print(f"MlFlow: Se produjo el siguiente error : {ex} ")

        @app.post("/")
        async def score(self, item: Item):
            try:
                #payload = await request.json()
                payload = item.dict()
                df = pd.DataFrame([payload])
                score = float(self.model.predict_proba(df)[0][1])
                return {"cod": 200, "msg": "Score calculated successfully.", "data": {"em_eventprobability": score}, "errors": None }
            except Exception as e:
                #logger.error(f"Error in import prediction: {e}")
                return {"cod": 500, "msg": "Error calculating score", "data": None, "errors": str(e)}
except Exception as ex:
    print(f"Desployment y Ingress: Se produjo el siguiente error : {ex} ")

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Desplegar el Modelo</strong></h4>
</div>

In [ ]:
try:
    ModelMlFlow.deploy()
    print("Modelo Desplegado")
except Exception as ex:
    serve.get_deployment("Iris_LogReg_Model").delete()
    print(f"Ray: Se produjo el siguiente error : {ex} ")

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>FastAPI genera automáticamente documentación interactiva en /docs</strong></h4>
</div>

http://localhost:8000/v1/iris-logreg-modelo/score/docs

<div style="background-color:AliceBlue; padding:10px; border-left:5px solid #2e8b57; margin-bottom:10px;">
  <h4 style="font-size:13px;">🤖 <strong>Verifica que Modelos estan Desplegados</strong></h4>
</div>

In [ ]:
serve.list_deployments()

In [ ]:
#serve.get_deployment("contrab-carga-iamc").delete()